In [249]:
%%writefile exaustiva_thrust.cu
#include <cmath>
#include <algorithm> 
#include <iostream>
#include <random>
#include <chrono>
#include <stdlib.h> 
#include <random>
#include <chrono>
#include <fstream>
#include <bitset>
#include <stack>
#include <utility>
#include <map>
#include <ctime>
#include <omp.h>
 // imports do thrust
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/functional.h>
#include <thrust/copy.h> 
using std::vector;
using std::cin;
using std::cout;
using std::endl;
using std::bitset;
using std::map;
using std::stack;
using std::pair;
using std::make_pair;
int main(){
    int N, M;
    cin >> N >> M;
    cout << N << endl;
    cout << M << endl;
    thrust::host_vector<int>filmes_por_categoria_cpu(M+1);
    thrust::host_vector<int> start_times_cpu(N+1);
    thrust::host_vector<int> end_times_cpu(N+1);
    thrust::host_vector<int> categories_cpu(N+1);

    for (int i = 1; i < M+1; i++){
        cin >> filmes_por_categoria_cpu[i];
    }

    for (int i = 1; i < N+1; i++){
        cin >> start_times_cpu[i];
        cin >> end_times_cpu[i];
        cin >> categories_cpu[i];
        if (start_times_cpu[i] == 0){
            start_times_cpu[i] = 24;
        }
        if (end_times_cpu[i] == 0){
            end_times_cpu[i] = 24;
        }
        if (start_times_cpu[i] < 0){
            continue;
        }
        if (end_times_cpu[i] < 0){
            continue;
        }
    }

    thrust::device_vector<int> start_times_gpu(start_times_cpu);
    thrust::device_vector<int> end_times_gpu(end_times_cpu);
    thrust::device_vector<int> categories_gpu(categories_cpu);
    thrust::device_vector<int> filmes_por_categoria_gpu(filmes_por_categoria_cpu);


    thrust::device_vector<int> dp((N+1)*(M+1), 0);

    // Inicializar a primeira linha da matriz com zeros
    thrust::fill(dp.begin(), dp.begin() + M+1, 0);

    // Preencher a matriz com as soluções para subproblemas menores
    // Preencher a matriz com as soluções para subproblemas menores
    for (int i = 1; i <= N; i++) {
        for (int j = 1; j <= M; j++) {
            // Encontrar o número máximo de filmes que podem ser assistidos até o filme i e categoria j
            int max_count = 0;
            for (int k = 0; k <= i; k++) {
              cout << "MATRIZ: " << i  << ", " << j << " K: " << k << " I : " << i << " J : " << j << endl;
              cout << start_times_gpu[k] << endl;
              cout << end_times_gpu[k] << endl;
              cout << "Categories " << categories_gpu[k] << " " << " End : " << end_times_gpu[k] << " Start : " << start_times_gpu[i] << " Dp: " << dp[(k*(M+1)) + j - 1] + 1 << " L : " << filmes_por_categoria_gpu[j] << endl;
              if (categories_gpu[k] == j && end_times_gpu[k] <= start_times_gpu[i] && dp[(k*(M + 1)) + j - 1] + 1 <= filmes_por_categoria_gpu[j]) {
                  cout << "Entrei no if e aqui está: " << dp[(k*(M+1)) + j - 1] + 1 << " " << max_count << endl;
                  max_count = max(max_count, dp[(k*(M+1)) + j - 1] + 1);
              } else {
                  cout << "Entrei no else e aqui está: " << dp[(k*(M)) + j] << " "  << max_count << endl;
                  max_count = max(max_count, dp[(k*(M+1)) + j]);
              }
            }
            dp[i*(M + 1) + j] = max_count;
            cout <<  "----------------------------------------------------" << endl;
        }
    }

    // Encontrar o número máximo de filmes que podem ser assistidos
    int max_count = 0;
    for (int j = 1; j <= M; j++) {
      max_count = max(max_count, dp[(N*(M+1)) + j]);
    }




    cout << max_count << endl;

    for (int i = 1; i < (N+1)*(M+1); i++) {
        cout << dp[i] << " ";
    }



    return 0;
}

Overwriting exaustiva_thrust.cu


In [250]:
!nvcc exaustiva_thrust.cu -o proj

In [251]:
!./proj < input4.txt

4
2
MATRIZ: 1, 1 K: 0 I : 1 J : 1
0
0
Categories 0  End : 0 Start : 1 Dp: 1 L : 1
Entrei no else e aqui está: 0 0
MATRIZ: 1, 1 K: 1 I : 1 J : 1
1
2
Categories 1  End : 2 Start : 1 Dp: 1 L : 1
Entrei no else e aqui está: 0 0
----------------------------------------------------
MATRIZ: 1, 2 K: 0 I : 1 J : 2
0
0
Categories 0  End : 0 Start : 1 Dp: 1 L : 3
Entrei no else e aqui está: 0 0
MATRIZ: 1, 2 K: 1 I : 1 J : 2
1
2
Categories 1  End : 2 Start : 1 Dp: 1 L : 3
Entrei no else e aqui está: 0 0
----------------------------------------------------
MATRIZ: 2, 1 K: 0 I : 2 J : 1
0
0
Categories 0  End : 0 Start : 3 Dp: 1 L : 1
Entrei no else e aqui está: 0 0
MATRIZ: 2, 1 K: 1 I : 2 J : 1
1
2
Categories 1  End : 2 Start : 3 Dp: 1 L : 1
Entrei no if e aqui está: 1 0
MATRIZ: 2, 1 K: 2 I : 2 J : 1
3
4
Categories 2  End : 4 Start : 3 Dp: 1 L : 1
Entrei no else e aqui está: 0 1
----------------------------------------------------
MATRIZ: 2, 2 K: 0 I : 2 J : 2
0
0
Categories 0  End : 0 Start : 3 Dp:

In [264]:
%%writefile exaustiva_thrust_copy.cu
#include <cmath>
#include <algorithm> 
#include <iostream>
#include <random>
#include <chrono>
#include <stdlib.h> 
#include <random>
#include <chrono>
#include <fstream>
#include <bitset>
#include <stack>
#include <utility>
#include <map>
#include <ctime>
#include <omp.h>
 // imports do thrust
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/functional.h>
#include <thrust/copy.h> 
using std::vector;
using std::cin;
using std::cout;
using std::endl;
using std::bitset;
using std::map;
using std::stack;
using std::pair;
using std::make_pair;
int main(){
    int N, M;
    cin >> N >> M;
    cout << N << endl;
    cout << M << endl;
    thrust::host_vector<int>filmes_por_categoria_cpu(M);
    thrust::host_vector<int> start_times_cpu(N);
    thrust::host_vector<int> end_times_cpu(N);
    thrust::host_vector<int> categories_cpu(N);

    for (int i = 0; i < M; i++){
        cin >> filmes_por_categoria_cpu[i];
    }

    for (int i = 0; i < N; i++){
        cin >> start_times_cpu[i];
        cin >> end_times_cpu[i];
        cin >> categories_cpu[i];
        if (start_times_cpu[i] == 0){
            start_times_cpu[i] = 24;
        }
        if (end_times_cpu[i] == 0){
            end_times_cpu[i] = 24;
        }
        if (start_times_cpu[i] < 0){
            continue;
        }
        if (end_times_cpu[i] < 0){
            continue;
        }
    }

    thrust::device_vector<int> start_times_gpu(start_times_cpu);
    thrust::device_vector<int> end_times_gpu(end_times_cpu);
    thrust::device_vector<int> categories_gpu(categories_cpu);
    thrust::device_vector<int> filmes_por_categoria_gpu(filmes_por_categoria_cpu);


    thrust::device_vector<int> dp((N+1) *(M+1), 0);

    // Inicializar a primeira linha da matriz com zeros
    thrust::fill(dp.begin(), dp.begin() + M+1, 0);

    // Preencher a matriz com as soluções para subproblemas menores
    // Preencher a matriz com as soluções para subproblemas menores
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < M; j++) {
            // Encontrar o número máximo de filmes que podem ser assistidos até o filme i e categoria j
            int max_count = 0;
            for (int k = 0; k <= i; k++) {
              cout << "MATRIZ: " << i  << ", " << j << " K: " << k << " I : " << i << " J : " << j << endl;
              cout << start_times_gpu[k] << endl;
              cout << end_times_gpu[k] << endl;
              cout << "Categories " << categories_gpu[k] << " " << " End : " << end_times_gpu[k] << " Start : " << start_times_gpu[i] << " Dp: " << dp[(k*(M)) + j] + 1 << " L : " << filmes_por_categoria_gpu[j] << endl;
              if (k == i){
                max_count = max(max_count, dp[(k*(M)) + j] + 1);
              }
              else if (categories_gpu[k] == j+1 && end_times_gpu[k] <= start_times_gpu[i] && dp[(k*(M)) + j] + 1 <= filmes_por_categoria_gpu[j]) {
                  cout << "Entrei no if e aqui está: " << dp[(k*(M)) + j] + 1 << " " << max_count << endl;
                  max_count = max(max_count, dp[(k*(M)) + j] + 1);
              } else {
                  cout << "Entrei no else e aqui está: " << dp[(k*(M)) + j] << " "  << max_count << endl;
                  max_count = max(max_count, dp[(k*(M)) + j + 1]);
              }
            }
            dp[(i*M) + j ] = max_count;
        }
    }

    // Encontrar o número máximo de filmes que podem ser assistidos
    int max_count = 0;
    for (int j = 0; j < M+1; j++) {
      max_count = max(max_count, dp[(N)*(M+1) + j]);
    }




    cout << max_count << endl;

    for (int i = 0; i < (N+1)*(M+1); i++) {
        cout << dp[i] << " ";
    }



    return 0;
}

Overwriting exaustiva_thrust_copy.cu


In [265]:
!nvcc exaustiva_thrust_copy.cu -o copy

In [266]:
!./copy < input4.txt

4
2
MATRIZ: 0, 0 K: 0 I : 0 J : 0
1
2
Categories 1  End : 2 Start : 1 Dp: 1 L : 1
MATRIZ: 0, 1 K: 0 I : 0 J : 1
1
2
Categories 1  End : 2 Start : 1 Dp: 1 L : 3
MATRIZ: 1, 0 K: 0 I : 1 J : 0
1
2
Categories 1  End : 2 Start : 3 Dp: 2 L : 1
Entrei no else e aqui está: 1 0
MATRIZ: 1, 0 K: 1 I : 1 J : 0
3
4
Categories 2  End : 4 Start : 3 Dp: 1 L : 1
MATRIZ: 1, 1 K: 0 I : 1 J : 1
1
2
Categories 1  End : 2 Start : 3 Dp: 2 L : 3
Entrei no else e aqui está: 1 0
MATRIZ: 1, 1 K: 1 I : 1 J : 1
3
4
Categories 2  End : 4 Start : 3 Dp: 1 L : 3
MATRIZ: 2, 0 K: 0 I : 2 J : 0
1
2
Categories 1  End : 2 Start : 5 Dp: 2 L : 1
Entrei no else e aqui está: 1 0
MATRIZ: 2, 0 K: 1 I : 2 J : 0
3
4
Categories 2  End : 4 Start : 5 Dp: 2 L : 1
Entrei no else e aqui está: 1 1
MATRIZ: 2, 0 K: 2 I : 2 J : 0
5
8
Categories 2  End : 8 Start : 5 Dp: 1 L : 1
MATRIZ: 2, 1 K: 0 I : 2 J : 1
1
2
Categories 1  End : 2 Start : 5 Dp: 2 L : 3
Entrei no else e aqui está: 1 0
MATRIZ: 2, 1 K: 1 I : 2 J : 1
3
4
Categories 2  End : 4 